2025: prepared 980 rows
getting data source for sandbox
✅ Done bulk-loading rate_curves from 2025-02-25 through 2025-06-05
🏃 View run bustling-robin-731 at: http://127.0.0.1:8768/#/experiments/1500/runs/6547b854ab8e415d991a7babf6cb5644
🧪 View experiment at: http://127.0.0.1:8768/#/experiments/1500
